In [ ]:
import pandas as pd
import numpy as np
import glob
import os

In [ ]:
# 자치구 별 유동인구들의 총합을 구하고 '자치구 별 유동인구 수_합계' 파일로 저장
# CSV 파일이 들어 있는 디렉토리 경로
directory_path = './cheonan/seoul'

# CSV 파일 목록 얻기
csv_files = glob.glob(os.path.join(directory_path, '*.csv'))

for csv_file in csv_files:
    df = pd.read_csv(csv_file)  # CSV 파일을 데이터프레임으로 읽기
    sum_of_last_two_columns = df.iloc[:, -2:].sum().sum()  # 마지막 두 열(서울시 자치구 별 유동인구)의 합 계산
    
    # 새로운 파일명 생성
    filename, extension = os.path.splitext(os.path.basename(csv_file))
    new_filename = f"{filename}_합계{extension}"
    
    # '자치구 별 유동인구 수_합계'를 파일명으로 갖는 새로운 CSV 파일 생성
    new_df = pd.DataFrame({'합계': [sum_of_last_two_columns]})
    new_df.to_csv(new_filename, index=False)
    
    print(f"{csv_file}의 마지막 두 열 합을 {new_filename}으로 저장하였습니다.")

In [ ]:
# 자치구 별 유동인구를 region과 float_pop을 컬럼으로 하여 './seoul_sum/floating_pop.csv'로 저장
# CSV 파일이 들어 있는 디렉토리 경로
directory_path = './seoul_sum'

# CSV 파일 목록 얻기
csv_files = glob.glob(os.path.join(directory_path, '*.csv'))

data_frames = []  # 데이터프레임들을 저장할 리스트

for csv_file in csv_files:
    df = pd.read_csv(csv_file)  # CSV 파일을 데이터프레임으로 읽기
    filename = os.path.basename(csv_file)
    
    # 파일명을 공백으로 분리하여 두 번째 문자를 첫 번째 컬럼으로(각 자치구를 이름으로) 변경
    filename_parts = filename.split()
    if len(filename_parts) > 1:
        df['새로운 컬럼'] = filename_parts[1]
    
    data_frames.append(df)  # 데이터프레임을 리스트에 추가

# data_frames 리스트에 있는 모든 데이터프레임을 하나로 합치기
merged_df = pd.concat(data_frames, ignore_index=True)

merged_df.columns=['float_pop', 'region']
# 합쳐진 데이터프레임을 'floating_pop'으로 저장
output_file_path = os.path.join(directory_path, './cheonan/pdreprocessed_data/floating_pop.csv')
merged_df.to_csv(output_file_path, index=False)

print("결과를 CSV 파일로 저장하였습니다.")

In [ ]:
# '따릉이대여소정보(23.06).xlsx' 파일에서 대여 방식 별 대여소 개수를 더하고 대여소 명에서 자치구 이름만 추출
# 엑셀 파일 경로
excel_file_path = './cheonan/따릉이대여소정보(23.06).xlsx'

# 엑셀 파일을 DataFrame으로 읽어들임
df = pd.read_excel(excel_file_path)

# NaN 값을 0으로 치환
df.fillna(0, inplace=True)

# 대여 방식 별 대여소의 개수(뒤에서 두 번째와 세 번째 열)를 합쳐서 새로운 컬럼 추가
df['대여 방식 별 대여소의 개수의 합'] = df.iloc[:, -3] + df.iloc[:, -2]

# 자치구 이름과 대여소 개수의 합(세 번째 열과 마지막 열)을 선택
selected_columns = df.iloc[:, [2, -1]]

# 선택한 열을 CSV 파일로 저장
output_csv_file_path = './cheoan/result.csv'
selected_columns.to_csv(output_csv_file_path, index=False)

print("CSV 파일로 저장하였습니다.")


In [ ]:
# 'result.csv' 파일에서 자치구 별 대여소 개수를 자치구 이름으로 그룹화
# CSV 파일 경로
csv_file_path = './result.csv'

# CSV 파일을 DataFrame으로 읽어들임
df = pd.read_csv(csv_file_path)

# result.csv 파일의 컬럼 설정 (자치구와 대여소 개수)
df.columns=['region', 'cnt']

# 'region' 열을 기준으로 'cnt' 값을 합쳐서 그룹별 합계 계산
grouped_sum = df.groupby('region')['cnt'].sum().reset_index()

# 결과를 새로운 CSV 파일로 저장
grouped_sum.to_csv('./cheonan/prerocessed_data/cnt.csv', index=False)

print("그룹화된 결과를 CSV 파일로 저장하였습니다.")

In [ ]:
# '인구밀도_20230806203002.xlsx' 파일에서 자치구 별 인구를 필터링하여 저장
# 엑셀 파일 경로
excel_file_path = './cheonan/인구밀도_20230806203002.xlsx'

# 엑셀 파일을 DataFrame으로 읽어들임
df = pd.read_excel(excel_file_path)

# 두 번째 컬럼이 NaN이 아닌 행들만 선택
# 자치구 별 소계가 나와있기 때문에 두 번째 컬럼이 NaN이 아닌 행들만 계산
filtered_df = df[df.iloc[:, 1].notna()]

# 두 번째(자치구 명)와 네 번째 열(인구수)만 선택
selected_columns = filtered_df.iloc[:, [1, 3]]

# 결과를 CSV 파일로 저장
output_csv_file_path = './cheonan/selected_result.csv'
selected_columns.to_csv(output_csv_file_path, index=False)

print("필터링된 결과를 CSV 파일로 저장하였습니다.")

In [ ]:
# 'selected_result.csv' 파일에서 자치구 별 인구수 추출
# CSV 파일 경로
csv_file_path = './cheonan/selected_result.csv'

# CSV 파일을 DataFrame으로 읽고, 의미없는 첫 번째 행은 스킵
df = pd.read_csv(csv_file_path, skiprows=[0])

#각 컬럼을 region과 pop으로 수정
df.columns=['region', 'pop']

# 결과를 CSV 파일로 저장
output_csv_file_path = './cheonan/prerocessed_data/pop.csv'
df.to_csv(output_csv_file_path, index=False)

print("선택된 결과를 CSV 파일로 저장하였습니다.")

In [ ]:
#전처리한 데이터들을 하나로 병합
# CSV 파일 경로들
csv_file_paths = ['./cheonan/preprocessed_data/pop.csv', './cheonan/preprocessed_data/floating_pop.csv', './cheonan/preprocessed_data/cnt.csv']

# CSV 파일들을 각각 DataFrame으로 읽어들임
dfs = [pd.read_csv(file_path) for file_path in csv_file_paths]

# 'region' 열을 기준으로 merge
merged_df = dfs[0]  # 첫 번째 DataFrame을 기준으로 설정
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on='region', how='outer')

# 결과를 CSV 파일로 저장
output_csv_file_path = './cheonan/preprocessed_data/merged_result.csv'
merged_df.to_csv(output_csv_file_path, index=False)

print("Merge된 결과를 CSV 파일로 저장하였습니다.")

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# 전처리된 데이터들을 이용하여 회귀 분석
csv_file_path = './cheonan/preprocessed_data/merged_result.csv'
df = pd.read_csv(csv_file_path)
# 종속 변수의 값은 대여소의 개수
y_col = ["cnt"]
# 단독 변수의 값은 유동 인구와 인구
x_cols = ["float_pop","pop"]

X = df[x_cols].values
y = df[y_col].values
reg = LinearRegression().fit(X, y)
print(reg.predict(X))

model = sm.OLS.from_formula("cnt ~ float_pop+pop", data=df)
result = model.fit()
result.predict(df[x_cols])
print(result.summary())